# Proyecto 9: Aprendizaje supervisado

## Descripcion del proyecto

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.
Crea un modelo con el máximo valor F1 posible. 

Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 
Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

### Instrucciones del proyecto 

- Descarga y prepara los datos.  Explica el procedimiento.

- Examina el equilibrio de clases. Entrena el modelo sin tener en cuenta el desequilibrio. Describe brevemente tus hallazgos.

- Mejora la calidad del modelo. Asegúrate de utilizar al menos dos enfoques para corregir el desequilibrio de clases. Utiliza conjuntos de entrenamiento y validación para encontrar el mejor modelo y el mejor conjunto de parámetros. Entrena diferentes modelos en los conjuntos de entrenamiento y validación. Encuentra el mejor. Describe brevemente tus hallazgos.

- Realiza la prueba final.

### Descripcion de los datos

Puedes encontrar los datos en el archivo  /datasets/Churn.csv file. Descarga el conjunto de datos.

Características

- RowNumber: índice de cadena de datos
- CustomerId: identificador de cliente único
- Surname: apellido
- CreditScore: valor de crédito
- Geography: país de residencia
- Gender: sexo
- Age: edad
- Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- Balance: saldo de la cuenta
- NumOfProducts: número de productos bancarios utilizados por el cliente
- HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- IsActiveMember: actividad del cliente (1 - sí; 0 - no)
- EstimatedSalary: salario estimado

Objetivo
- Exited: El cliente se ha ido (1 - sí; 0 - no)

## Evaluacion del proyecto

Hemos definido los criterios de evaluación para el proyecto. Lee esto con atención antes de pasar al ejercicio.

Esto es lo que los revisores buscarán cuando evalúen tu proyecto:

- ¿Cómo preparaste los datos para el entrenamiento? ¿Procesaste todos los tipos de características?
- ¿Explicaste los pasos de preprocesamiento lo suficientemente bien?
- ¿Cómo investigaste el equilibrio de clases?
- ¿Estudiaste el modelo sin tener en cuenta el desequilibrio de clases?
- ¿Qué descubriste sobre la investigación del ejercicio?
- ¿Dividiste correctamente los datos en conjuntos?
- ¿Cómo trabajaste con el desequilibrio de clases?
- ¿Utilizaste al menos dos técnicas para corregir el desequilibrio?
- ¿Realizaste correctamente el entrenamiento, la validación y las pruebas finales del modelo?
- ¿Qué tan alto es tu valor F1?
- ¿Examinaste los valores AUC-ROC?
- ¿Mantuviste la estructura del proyecto y el código limpio?
Ya tienes las hojas informativas y los resúmenes de capítulos, tienes todo para continuar con el proyecto.

¡Buena suerte!

## Preparacion de datos

In [1]:
#Importacion de librerias 
# Importar librerias necesarias para el proyecto
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from IPython.display import HTML
from sklearn.metrics import f1_score

In [2]:
# Cargar los datos
data = pd.read_csv("datasets/Churn.csv")

# Realizar limpieza si es necesario (eliminar columnas irrelevantes, tratar valores faltantes, etc.)
# Por ejemplo, podrías considerar eliminar 'RowNumber', 'CustomerId', y 'Surname' ya que probablemente no sean relevantes para la predicción
data_clean = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Importar Archivos

# Mostrar datos relevantes del dataframe
display(HTML('<hr>'))
display(HTML('<h1> Clientes de Beta-bank '))
display(data_clean.head())
display(data_clean.info())
display(data_clean.describe())
display(HTML('<hr>'))



# Verificacion de datos nulos 
nulos = data_clean.isnull().sum()
display(HTML('<h2> Verificacion de valores nulos'))
display(nulos)

# Verificación de valores duplicados
duplicados = data_clean.duplicated().sum()
display(HTML('<h2> Verificación de valores duplicados </h2>'))
display(f"Total de valores duplicados: {duplicados}")

display(HTML('<hr>'))


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           9091 non-null   float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


None

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

'Total de valores duplicados: 0'

## Modelado sin tener en cuenta el desequilibrio de clases

In [3]:
# Primero examinamos el equilibrio de clases

print(data_clean['Exited'].value_counts(normalize=True))


Exited
0    0.7963
1    0.2037
Name: proportion, dtype: float64


In [4]:
X = data_clean.drop('Exited', axis=1)
y = data_clean['Exited']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Predecir y evaluar
predictions = model.predict(X_test)
print(f'F1 Score: {f1_score(y_test, predictions)}')

ValueError: could not convert string to float: 'France'

Descripción del proyecto:
Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.
Necesitamos predecir si un cliente dejará el banco pronto. Tú tienes los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.
Crea un modelo con el máximo valor F1 posible. 
Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 
Además, debes medir la métrica AUC-ROC y compararla con el valor F1.
Instrucciones del proyecto:
- Descarga y prepara los datos.  Explica el procedimiento.
- Examina el equilibrio de clases. Entrena el modelo sin tener en cuenta el desequilibrio. Describe brevemente tus hallazgos.
- Mejora la calidad del modelo. Asegúrate de utilizar al menos dos enfoques para corregir el desequilibrio de clases. Utiliza conjuntos de entrenamiento y validación para encontrar el mejor modelo y conjunto de parámetros. Entrena diferentes modelos en los conjuntos de entrenamiento y validación. Encuentra el mejor. Describe brevemente tus hallazgos.
- Realiza la prueba final.
Descripcion de datos
Puedes encontrar los datos en el archivo  /datasets/Churn.csv file. Descarga el conjunto de datos.

Características

- RowNumber: índice de cadena de datos
- CustomerId: identificador de cliente único
- Surname: apellido
- CreditScore: valor de crédito
- Geography: país de residencia
- Gender: sexo
- Age: edad
- Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- Balance: saldo de la cuenta
- NumOfProducts: número de productos bancarios utilizados por el cliente
- HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- IsActiveMember: actividad del cliente (1 - sí; 0 - no)
- EstimatedSalary: salario estimado
Objetivo
- Exited: El cliente se ha ido (1 - sí; 0 - no)
